In [2]:
import pandas as pd
import numpy as np 
import json
import glob 
import ast 
import gzip
import pyarrow as pa
import pyarrow.parquet as pq

%load_ext autoreload
%autoreload 2

**CONVERTIR DATOS ANIDADOS DE UN JSON EN UN DATAFRAME CON PANDAS EN PYTHON**

In [3]:
v_general = [] #Se crea esta lista para almacenar los objetos python resultantes

#Se abre el archivo JSON comprimido ('user_reviews.json.gz') utilizando la biblioteca gzip. 
#Este archivo es procesado línea por línea en el siguiente bucle for.

for i in gzip.open('C:\\Users\\ACER\\Documents\\HENRY\\LABS\\PI01\\DATASETS\\user_reviews.json.gz'): 
    v_general.append(ast.literal_eval(i.decode('utf-8'))) 
    
#En cada iteración del bucle for, se realiza lo siguiente:
#La línea del archivo se decodifica desde su formato comprimido en UTF-8 para convertirla en una cadena de texto legible.
#Luego, se utiliza la función ast.literal_eval() para interpretar la cadena como una estructura de datos Python. 
#Esto es útil para convertir cada línea del archivo JSON en un objeto Python, como un diccionario o una lista, y almacenarlo en la lista v_general.
#Se crea un nuevo DataFrame a partir de la lista de datos v_general.
reviews = pd.DataFrame(v_general)

reviews


,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


**EXPLORACIÓN DE DATA**

In [4]:
#Se obtine la información sobre los tipos de datos que hay en el DataFrame
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [5]:
reviews.describe()

,user_id,user_url,reviews
count,25799,25799,25799
unique,25485,25485,25459
top,76561198027488037,http://steamcommunity.com/profiles/76561198027...,[]
freq,3,3,28


In [6]:
reviews.dtypes

user_id     object
user_url    object
reviews     object
dtype: object

In [7]:
reviews.columns

Index(['user_id', 'user_url', 'reviews'], dtype='object')

In [8]:
reviews.shape

(25799, 3)

### La columna 'reviews', que contiene una lista de diccionarios anidados, se descompone para que cada elemento de la lista se convierta en una fila individual en el conjunto de datos.

In [9]:
df_reviews_des = reviews.explode('reviews')
df_reviews_des

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...,...
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ..."
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


In [10]:
reviews_norm = pd.json_normalize(df_reviews_des['reviews'].dropna())
reviews.head(5)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [11]:
reviews_norm.reset_index(inplace=True)
reviews_norm

,index,funny,posted,last_edited,item_id,helpful,recommend,review
0,0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...
59300,59300,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
59301,59301,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
59302,59302,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59303,59303,,Posted July 20.,,730,No ratings yet,True,:D


In [12]:
df_reviews_des.reset_index(inplace=True)
df_reviews_des

,index,user_id,user_url,reviews
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
2,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
3,1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
4,1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...,...,...
59328,25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
59329,25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
59330,25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ..."
59331,25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


CONCATENAR EL DATA ORIGINAL Y ELIMINAR LA COLUMNA ORIGINAL 'reviews' ANIDADA Y ADICIONALMENTE SE ELIMINA EL DOBLE INDEX PARA QUE UNICAMENTE QUEDE UNA FORMA DE ORDENAR POR INDICE 

In [13]:
reviews_2 = pd.concat([df_reviews_des, reviews_norm], axis=1)
reviews_2 = reviews_2.drop(columns=['reviews'])
reviews_2 = reviews_2.drop(columns="index")
reviews_2

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,NaN,NaN,NaN,NaN,NaN,NaN,NaN


SE VERIFICAN SI EXISTEN DATOS DUPLICADOS Y SE PROCEDE A ELIMINARLOS 

In [14]:
DataType_rev = {"columna":[],"DataType":[]} #genero un diccionario vacio para ir almacenando lo que genere el bucle

for columna in reviews_2.columns: #un bucle que va recorriendo 
    DataType_rev["columna"].append(columna)
    DataType_rev["DataType"].append(reviews_2[columna].apply(type).unique())

Analyst_rev= pd.DataFrame(DataType_rev)
Analyst_rev

,columna,DataType
0,user_id,[<class 'str'>]
1,user_url,[<class 'str'>]
2,funny,"[<class 'str'>, <class 'float'>]"
3,posted,"[<class 'str'>, <class 'float'>]"
4,last_edited,"[<class 'str'>, <class 'float'>]"
5,item_id,"[<class 'str'>, <class 'float'>]"
6,helpful,"[<class 'str'>, <class 'float'>]"
7,recommend,"[<class 'bool'>, <class 'float'>]"
8,review,"[<class 'str'>, <class 'float'>]"


In [15]:
revs_dups= reviews_2.loc[reviews_2.duplicated()]
revs_dups

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
1112,bobseagull,http://steamcommunity.com/id/bobseagull,,"Posted September 24, 2015.",,346110,1 of 1 people (100%) found this review helpful,True,yep
2894,ImSeriouss,http://steamcommunity.com/id/ImSeriouss,,"Posted January 13, 2014.",,211820,No ratings yet,True,If you want to play this game.. expect glithes...
2895,ImSeriouss,http://steamcommunity.com/id/ImSeriouss,,"Posted January 10, 2014.",,440,No ratings yet,True,Really good game! fun! Good for people who wan...
2896,ImSeriouss,http://steamcommunity.com/id/ImSeriouss,,"Posted March 19, 2012.",,42680,No ratings yet,True,Good but a bit overdone. Still love it though.
3582,76561198062039159,http://steamcommunity.com/profiles/76561198062...,,"Posted December 11, 2015.",,730,0 of 1 people (0%) found this review helpful,True,I rate it R8/Revolver
...,...,...,...,...,...,...,...,...,...
59327,76561198312638244,http://steamcommunity.com/profiles/76561198312...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
dups_del_rev = reviews_2.drop_duplicates(keep='first')
dups_del_rev

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...,...
59323,76561198306599751,http://steamcommunity.com/profiles/76561198306...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59324,Ghoustik,http://steamcommunity.com/id/Ghoustik,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59325,76561198310819422,http://steamcommunity.com/profiles/76561198310...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59326,76561198312638244,http://steamcommunity.com/profiles/76561198312...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
reviews_2.shape

(59333, 9)

SE PROCEDE A CORREGIR LA FECHA EN 'posted' POR EL FORMATO CORRECTO YYYY-MM-DD 

In [18]:
reviews_2['posted'] = reviews_2['posted'].str.replace(r'Posted', '').str.strip()
reviews_2['posted'] = pd.to_datetime(reviews_2['posted'], errors='coerce')
reviews_2

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-11-05,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-07-15,,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-04-21,,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,2014-06-24,,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,2013-09-08,,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,NaN,NaT,NaN,NaN,NaN,NaN,NaN
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,NaN,NaT,NaN,NaN,NaN,NaN,NaN
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,NaN,NaT,NaN,NaN,NaN,NaN,NaN
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,NaN,NaT,NaN,NaN,NaN,NaN,NaN


VERIFICACIÓN DE DATOS NULOS Y ELIMINACIÓN DE LOS MISMOS 



In [19]:
#Se verifica la cantidad de datos nulos que hay por columna y se suman los datos 
nulls = reviews_2.isnull().sum()
nulls

user_id            0
user_url           0
funny             28
posted         10147
last_edited       28
item_id           28
helpful           28
recommend         28
review            28
dtype: int64

In [20]:
reviews_2 = reviews_2.dropna().reset_index(drop=True)
reviews_2

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-11-05,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-07-15,,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-04-21,,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,2014-06-24,,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,2013-09-08,,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...,...
49181,llDracuwulf,http://steamcommunity.com/id/llDracuwulf,1 person found this review funny,2015-10-14,,730,1 of 1 people (100%) found this review helpful,True,its FUNNNNNNNN
49182,76561198223837952,http://steamcommunity.com/profiles/76561198223...,,2015-10-10,,253980,No ratings yet,True,Awesome fantasy game if you don't mind the gra...
49183,76561198229845636,http://steamcommunity.com/profiles/76561198229...,,2015-10-31,,730,No ratings yet,True,Prettyy Mad Game
49184,76561198232478272,http://steamcommunity.com/profiles/76561198232...,,2015-12-14,,730,No ratings yet,True,AMAZING GAME 10/10


**CARGA DEL CONJUNTO DE DATOS**

In [22]:
Save = 'data/user_reviews_clean.csv'
reviews_2.to_csv(Save, index=False, encoding='utf-8')

In [23]:
#Indico donde quiero guardar el parquet y con que nombre
output_file= "data/user_reviews_clean.parquet"

#Transformo a traves de una tabla el archivo csv en parquet
table = pa.Table.from_pandas(reviews_2)
pq.write_table(table,output_file)